# Take Control of GPT : Use the API. Don't let the API use you

### My first openai API Project!!!

Here, I use the openai Python lib and some custom classes to handle processing prompts and responses. I store the results in a pandas dataframe, then format the code in a pandas 'style' object. I can use the dataframe and other objects (like the multiline string of prompts) to manage context for later prompts. Yes, context from prompts and answers. So much more to do with this. But this is a start.

## Setup

In [1]:
%%html
<style>
    /* Enable word wrap */
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      white-space: nowrap;
      overflow: hidden;
      text-overflow: ellipsis;
    }
</style>

### Imports

In [2]:
# ...
import openai
import os
import pprint
import sys
from pprint import pprint
import json
import pyarrow as pa
import pandas as pd

lib = os.path.abspath(os.path.join('..', '..', 'libs'))
sys.path.append(lib)

from MessageParser import MessageParser
from ResponseParser import ResponseParser
from PromptLogs import PromptLogs


### Config

In [3]:
# ...
# *******************************************************************************************************
# SET THE API KEY
# ================
# -- set the env variable OPENAI_API_KEY to your OpenAI API key       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
openai.api_key = os.getenv("OPENAI_API_KEY")
# *******************************************************************************************************

# Setup up pandas properties
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line break in DataFrame
pd.set_option('display.max_colwidth', None)  # Display the full content of cells



## Main code

Prep the system prompts

In [4]:
sys_prompt_list = []
sys_prompt_1 = """You are a software expert."""
sys_prompt_2 = """Use. "```python" style for code blocks."""
sys_prompt_3 = """Include the name for a file where I would store individual code for functions or classes."""

sys_prompt_list.append(sys_prompt_1)
sys_prompt_list.append(sys_prompt_2)
sys_prompt_list.append(sys_prompt_3)

sys_prompt_string = "".join(sys_prompt_list)

### a PromptLogs object will hold all the individual individual parser objects, which hold the response from a single exchange

In [5]:
logs = PromptLogs()

Generate the message and make the call

In [6]:
message_list = f"""
system: {sys_prompt_string}
user: Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.
"""

# a MessageParser object will process the message_list and make an acceptable messages object
message_parser = MessageParser(message_list)
messages = message_parser.parse()

# Make the openai call and get the response
# todo: I'll abstract this away in the future
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)


A response parser will process the response

In [7]:
response_parser = ResponseParser(response=response)

response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Python code for a generating Fibonacci sequence using a function named \"```def fibonacci_sequence(n: int)```.\" This function will take an integer number \"n\" and then the function will generate the first \"n\" terms/binaries of the Fibonacci sequence storing it in a list object. Finally, the function will return the complete set of \"n\" terms.\n\n```python\n# File: fibonacci.py\n\ndef fibonacci_sequence(n: int):\n    \"\"\"Return a list of the first n \n    numbers in the Fibonacci sequence\"\"\"\n    fibo = [0, 1] # start with fibonacci \n    while len(fibo) < n:\n        fibo.append(fibo[-1] + fibo[-2])\n    return fibo\n```\n\n\nSecondly, here is how to write a function in Python that zero pads a string named \"```def zero_padding(name: str)```.\"  This function will receive a name, and then the function will compute and then return a length-10 version of valu

See a pandas dataframe of the logged response

In [8]:
response_parser.df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-756I3v00Sw8dzfNGldHFTe00t5PvN,1.681450e+09,gpt-3.5-turbo-0301,66.0,"#&nbsp;File:&nbsp;fibonacci.py<br><br>def&nbsp;fibonacci_sequence(n:&nbsp;int):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""Return&nbsp;a&nbsp;list&nbsp;of&nbsp;the&nbsp;first&nbsp;n&nbsp;<br>&nbsp;&nbsp;&nbsp;&nbsp;numbers&nbsp;in&nbsp;the&nbsp;Fibonacci&nbsp;sequence""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;fibo&nbsp;=&nbsp;[0,&nbsp;1]&nbsp;#&nbsp;start&nbsp;with&nbsp;fibonacci&nbsp;<br>&nbsp;&nbsp;&nbsp;&nbsp;while&nbsp;len(fibo)&nbsp;<&nbsp;n:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;fibo.append(fibo[-1]&nbsp;+&nbsp;fibo[-2])<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;fibo<br>#&nbsp;File:&nbsp;zero_pad.py<br><br>def&nbsp;zero_padding(name:&nbsp;str):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;Return&nbsp;a&nbsp;name&nbsp;into&nbsp;10-char&nbsp;spaced&nbsp;digit:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;padded&nbsp;to&nbsp;10&nbsp;spaces&nbsp;with&nbsp;leading&nbsp;0´s.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;name.zfill(10)&nbsp;#provides&nbsp;a&nbsp;zero-padded&nbsp;version&nbsp;from&nbsp;given&nbsp;string<br>","[\n# File: fibonacci.py\n\ndef fibonacci_sequence(n: int):\n """"""Return a list of the first n \n numbers in the Fibonacci sequence""""""\n fibo = [0, 1] # start with fibonacci \n while len(fibo) < n:\n fibo.append(fibo[-1] + fibo[-2])\n return fibo\n, \n# File: zero_pad.py\n\ndef zero_padding(name: str):\n """"""\n Return a name into 10-char spaced digit:\n padded to 10 spaces with leading 0´s.\n """"""\n return name.zfill(10) #provides a zero-padded version from given string\n]","Python code for a generating Fibonacci sequence using a function named ""```def fibonacci_sequence(n: int)```."" This function will take an integer number ""n"" and then the function will generate the first ""n"" terms/binaries of the Fibonacci sequence storing it in a list object. Finally, the function will return the complete set of ""n"" terms.\n\n```python\n# File: fibonacci.py\n\ndef fibonacci_sequence(n: int):\n """"""Return a list of the first n \n numbers in the Fibonacci sequence""""""\n fibo = [0, 1] # start with fibonacci \n while len(fibo) < n:\n fibo.append(fibo[-1] + fibo[-2])\n return fibo\n```\n\n\nSecondly, here is how to write a function in Python that zero pads a string named ""```def zero_padding(name: str)```."" This function will receive a name, and then the function will compute and then return a length-10 version of value-named string that will be padding with leading zeros to create 10 chars in total.\n\n```python\n# File: zero_pad.py\n\ndef zero_padding(name: str):\n """"""\n Return a name into 10-char spaced digit:\n padded to 10 spaces with leading 0´s.\n """"""\n return name.zfill(10) #provides a zero-padded version from given string\n```"


In [9]:
# A pandas style applies custom formatting to a DataFrame to make it easier to read and
# understand. It does not offer the full range of functionality that a DataFrame provides.
response_parser.styled_log

,id,created,model,usage,all_code
0,chatcmpl-756I3v00Sw8dzfNGldHFTe00t5PvN,1681450175.000000,gpt-3.5-turbo-0301,66.000000,"# File: fibonacci.pydef fibonacci_sequence(n: int): """"""Return a list of the first n numbers in the Fibonacci sequence"""""" fibo = [0, 1] # start with fibonacci while len(fibo) < n: fibo.append(fibo[-1] + fibo[-2]) return fibo# File: zero_pad.pydef zero_padding(name: str): """""" Return a name into 10-char spaced digit: padded to 10 spaces with leading 0´s. """""" return name.zfill(10) #provides a zero-padded version from given string"


In [10]:
# this is a PromptLogs object again. We'll add the response_parser to the logs
logs.append(messages, response_parser)
logs.styled_log

[{'role': 'system', 'content': 'You are a software expert.Use. "```python" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]


KeyError: 'sid'

## And we have another exchange

In [ ]:
message_list = f"""
system: {sys_prompt_string}
user: Write a python function that returns numbers 1-10 in a list. 
"""

message_parser = MessageParser(message_list)
messages = message_parser.parse()
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)
response_parser = ResponseParser(response=response)

logs.append(messages, response_parser)

response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Here's the `python` function that returns numbers 1-10 in a list:\n```python\ndef get_numbers():\n    return list(range(1, 11))\n```\n\nYou can store this code in a file called `functions.py` or any other preferred name as per your requirement or preference.",
        "role": "assistant"
      }
    }
  ],
  "created": 1681449538,
  "id": "chatcmpl-7567mnV8tgBTGmgzT4ZDD4uLvQTyV",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 62,
    "prompt_tokens": 60,
    "total_tokens": 122
  }
}
[{'role': 'system', 'content': 'You are a software expert.Use. "```python" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write a python function that returns numbers 1-10 in a list.'}]


In [ ]:
logs.df

,id,created,model,usage,all_code,code_blocks,content,sid,prompts
0,chatcmpl-7567b9uvhCdwBVYGIBPjJwUMFTpaX,1.681450e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci(num):<br>&nbsp;&nbsp;&nbsp;&nbsp;a,&nbsp;b&nbsp;=&nbsp;0,&nbsp;1<br>&nbsp;&nbsp;&nbsp;&nbsp;for&nbsp;i&nbsp;in&nbsp;range(num):<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;yield&nbsp;a<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;a,&nbsp;b&nbsp;=&nbsp;b,&nbsp;a&nbsp;+&nbsp;b<br>>>>&nbsp;for&nbsp;fib&nbsp;in&nbsp;fibonacci(10):<br>...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;print(fib)<br>...<br>0<br>1<br>1<br>2<br>3<br>5<br>8<br>13<br>21<br>34<br>def&nbsp;zero_pad(string):<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;string.zfill(10)<br>>>>&nbsp;padded_string&nbsp;=&nbsp;zero_pad('123')<br>>>>&nbsp;print(padded_string)<br>'0000000123'<br>","[\ndef fibonacci(num):\n a, b = 0, 1\n for i in range(num):\n yield a\n a, b = b, a + b\n, \n>>> for fib in fibonacci(10):\n... print(fib)\n...\n0\n1\n1\n2\n3\n5\n8\n13\n21\n34\n, \ndef zero_pad(string):\n return string.zfill(10)\n, \n>>> padded_string = zero_pad('123')\n>>> print(padded_string)\n'0000000123'\n]","Fibonacci sequence code:\n\n```python\ndef fibonacci(num):\n a, b = 0, 1\n for i in range(num):\n yield a\n a, b = b, a + b\n```\nExample:\n\n```python\n>>> for fib in fibonacci(10):\n... print(fib)\n...\n0\n1\n1\n2\n3\n5\n8\n13\n21\n34\n```\n\nZero padding function for string:\n\n```python\ndef zero_pad(string):\n return string.zfill(10)\n```\nExample:\n\n```python\n>>> padded_string = zero_pad('123')\n>>> print(padded_string)\n'0000000123'\n``` \n\nYou may store the Fibonacci function in a file named `fibonacci.py` and the zero padding function may go to a file named `string_utils.py`. Your directories would look like this:\n```\n├── fibonacci.py\n└── string_utils.py\n```",a1,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]"
0,chatcmpl-7567mnV8tgBTGmgzT4ZDD4uLvQTyV,1.681450e+09,gpt-3.5-turbo-0301,60.0,"def&nbsp;get_numbers():<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;list(range(1,&nbsp;11))<br>","[\ndef get_numbers():\n return list(range(1, 11))\n]","Here's the `python` function that returns numbers 1-10 in a list:\n```python\ndef get_numbers():\n return list(range(1, 11))\n```\n\nYou can store this code in a file called `functions.py` or any other preferred name as per your requirement or preference.",a2,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write a python function that returns numbers 1-10 in a list.'}]"


### And print the styled logs with the code from two separate prompt exchanges

In [ ]:
logs.styled_log

,id,created,model,usage,prompts,all_code
sid,,,,,,
a1,chatcmpl-7567b9uvhCdwBVYGIBPjJwUMFTpaX,1681449527.000000,gpt-3.5-turbo-0301,66.000000,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]","def fibonacci(num): a, b = 0, 1 for i in range(num): yield a a, b = b, a + b>>> for fib in fibonacci(10):... print(fib)...0112358132134def zero_pad(string): return string.zfill(10)>>> padded_string = zero_pad('123')>>> print(padded_string)'0000000123'"
a2,chatcmpl-7567mnV8tgBTGmgzT4ZDD4uLvQTyV,1681449538.000000,gpt-3.5-turbo-0301,60.000000,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write a python function that returns numbers 1-10 in a list.'}]","def get_numbers(): return list(range(1, 11))"
